In [91]:
import cv2
import numpy as np

# 讀取圖像
image = cv2.imread('input1.png')

# 將圖像轉換為HSV色彩空間
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 定義Cyan的範圍
lower_cyan = np.array([80, 100, 100])
upper_cyan = np.array([100, 255, 255])

# 定義Red的範圍
lower_red = np.array([0, 100, 100])
upper_red = np.array([10, 255, 255])

# 在範圍內創建掩模
mask_cyan = cv2.inRange(hsv, lower_cyan, upper_cyan)
mask_red = cv2.inRange(hsv, lower_red, upper_red)

# 進行形態學操作，以消除噪點
kernel = np.ones((5,5), np.uint8)
mask_cyan = cv2.morphologyEx(mask_cyan, cv2.MORPH_OPEN, kernel)
mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_OPEN, kernel)

# 在二值掩模中找到輪廓
contours_cyan, _ = cv2.findContours(mask_cyan, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 找到最大的輪廓，即玩家和最近的神龛
if len(contours_cyan) > 0:
    player_contour = max(contours_cyan, key=cv2.contourArea)
    player_center, player_radius = cv2.minEnclosingCircle(player_contour)
    player_center = tuple(map(int, player_center))

if len(contours_red) > 0:
    shrine_contour = max(contours_red, key=cv2.contourArea)
    shrine_center, shrine_radius = cv2.minEnclosingCircle(shrine_contour)
    shrine_center = tuple(map(int, shrine_center))

# 繪製玩家和神龛的圓圈
cv2.circle(image, player_center, int(player_radius), (255, 255, 0), 2)  # Cyan color for player
cv2.circle(image, shrine_center, int(shrine_radius), (0, 0, 255), 2)  # Red color for shrine

# 繪製黃色線條來表示玩家的朝向
angle = np.arctan2(shrine_center[1] - player_center[1], shrine_center[0] - player_center[0]) * 180 / np.pi
angle = angle if angle >= 0 else 360 + angle
cv2.line(image, player_center, (int(player_center[0] + player_radius * np.cos(angle * np.pi / 180)), 
                                 int(player_center[1] + player_radius * np.sin(angle * np.pi / 180))), 
         (0, 255, 255), 2)  # Yellow color for direction line

# 計算旋轉角度
rotation_angle = angle - 90  # Assuming player initially faces upward

# 繪製文字（旋轉角度）
cv2.putText(image, f'Rotation Angle: {rotation_angle:.2f} degrees', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
            0.7, (0, 255, 255), 2, cv2.LINE_AA)

# 顯示結果圖像
cv2.imshow('Result', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
